In [2]:
#Définition des librairies
library(jsonlite)
suppressPackageStartupMessages(library(Hmisc))
suppressPackageStartupMessages(library(cluster))
suppressPackageStartupMessages(library(Ecdat))
suppressPackageStartupMessages(library(compareGroups))
suppressPackageStartupMessages(library(randomForest))
suppressPackageStartupMessages(library(dplyr))

In [3]:
#Chargement du jeu de données
path="/home/user/Documents/STA211-Datamining/Projet_STA/"
Airbnb<-fromJSON(paste(path,"airbnb_V3.json",sep=""))
sapply(Airbnb,class)
summary(Airbnb)


_id           accommodates              bathrooms 
             "integer"            "character"            "character" 
              bed_type               bedrooms                   beds 
           "character"            "character"            "character" 
         property_type              room_type         arrondissement 
           "character"            "character"              "numeric" 
host_identity_verified      host_is_superhost          host_listings 
           "character"            "character"            "character" 
         host_response             host_since          cancel_policy 
           "character"            "character"            "character" 
      instant_bookable        availability_30       availability_365 
           "character"              "integer"              "integer" 
       availability_60        availability_90           first_review 
             "integer"              "integer"            "character" 
           last_review             nb_reviews    score_communication 
           "character"            "character"              "numeric" 
        score_location            score_other      reviews_per_month 
             "numeric"              "numeric"              "numeric" 
           total_price            price_level 
             "numeric"            "character"

      _id           accommodates        bathrooms           bed_type        
 Min.   :    2525   Length:41094       Length:41094       Length:41094      
 1st Qu.: 3688220   Class :character   Class :character   Class :character  
 Median : 7887020   Mode  :character   Mode  :character   Mode  :character  
 Mean   : 8425822                                                           
 3rd Qu.:13304907                                                           
 Max.   :17968815                                                           
   bedrooms             beds           property_type       room_type        
 Length:41094       Length:41094       Length:41094       Length:41094      
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            

In [4]:

character_vars <- lapply(Airbnb, class) == "character"
Airbnb[, character_vars] <- lapply(Airbnb[, character_vars], as.factor)
#définition de la variable arrondissement comme qualitative
Airbnb$arrondissement <- as.factor(Airbnb$arrondissement ) 
sapply(Airbnb,class)

_id           accommodates              bathrooms 
             "integer"               "factor"               "factor" 
              bed_type               bedrooms                   beds 
              "factor"               "factor"               "factor" 
         property_type              room_type         arrondissement 
              "factor"               "factor"               "factor" 
host_identity_verified      host_is_superhost          host_listings 
              "factor"               "factor"               "factor" 
         host_response             host_since          cancel_policy 
              "factor"               "factor"               "factor" 
      instant_bookable        availability_30       availability_365 
              "factor"              "integer"              "integer" 
       availability_60        availability_90           first_review 
             "integer"              "integer"               "factor" 
           last_review             nb_reviews    score_communication 
              "factor"               "factor"              "numeric" 
        score_location            score_other      reviews_per_month 
             "numeric"              "numeric"              "numeric" 
           total_price            price_level 
             "numeric"               "factor"

In [5]:
Airbnb_train <- Airbnb %>% sample_frac(0.3)
disMat<-daisy(Airbnb_train,metric = "gower")

In [19]:
set.seed(123)
mixedClusters<-kmeans(disMat, centers=3)

In [20]:
table(mixedClusters$cluster)


   1    2    3 
2746 3672 5910 

In [21]:
Airbnb_train$cluster<-mixedClusters$cluster
Airbnb_train$cluster <- as.factor(Airbnb_train$cluster ) 

In [22]:
sapply(Airbnb_train,class)

_id           accommodates              bathrooms 
             "integer"               "factor"               "factor" 
              bed_type               bedrooms                   beds 
              "factor"               "factor"               "factor" 
         property_type              room_type         arrondissement 
              "factor"               "factor"               "factor" 
host_identity_verified      host_is_superhost          host_listings 
              "factor"               "factor"               "factor" 
         host_response             host_since          cancel_policy 
              "factor"               "factor"               "factor" 
      instant_bookable        availability_30       availability_365 
              "factor"              "integer"              "integer" 
       availability_60        availability_90           first_review 
             "integer"              "integer"               "factor" 
           last_review             nb_reviews    score_communication 
              "factor"               "factor"              "numeric" 
        score_location            score_other      reviews_per_month 
             "numeric"              "numeric"              "numeric" 
           total_price            price_level                cluster 
             "numeric"               "factor"               "factor"

In [24]:
group<-compareGroups(cluster~accommodates+bathrooms+bed_type+bedrooms+beds+room_type
                     +host_identity_verified+host_listings+host_response+host_since+cancel_policy
                     +instant_bookable+availability_365+availability_90+availability_60+availability_30
                     +first_review+last_review+nb_reviews
                     +price_level+total_price
                     ,data = Airbnb_train)
clustab<-createTable(group)
clustab


--------Summary descriptives table by 'cluster'---------

________________________________________________________________________ 
                             1            2            3       p.overall 
                           N=2746       N=3672       N=5910              
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯ 
accommodates:                                                    0.000   
    T1                  195 (7.10%)   41 (1.12%)  270 (4.57%)            
    T2                  1686 (61.4%) 377 (10.3%)  3872 (65.5%)           
    T3                  279 (10.2%)  404 (11.0%)  557 (9.42%)            
    T4                  462 (16.8%)  1665 (45.3%) 1085 (18.4%)           
    T5+                 124 (4.52%)  1185 (32.3%) 126 (2.13%)            
bathrooms:                                                      <0.001   
    1-None               5 (0.18%)    1 (0.03%)    4 (0.07%)             
    2-One               2502 (91.1%) 2773 (75.5%) 570